In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import duckdb

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
con = duckdb.connect(database=":memory:")

In [ ]:
%run ./common_utils.ipynb

In [2]:
def add_new_gms(df):
    df_schd = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
    df_schd['Date'] = pd.to_datetime(df_schd.Date)

    df['game_id'] = df.Date.astype(str) + "_" + df.AwayABV + "_" + df.HomeABV
    df_schd['game_id'] = df_schd.Date.astype(str) + "_" + df_schd.AwayABV + "_" + df_schd.HomeABV
    retrieved = df_schd[(df_schd.rtrvd == 1)].game_id.unique().tolist()

    df['rtrvd'] = np.where(df.game_id.isin(retrieved), 1, 0)
    df = df.drop('game_id', axis=1)
    
    return df

In [9]:
last_reg_gm = {2025: "2026-04-12", 2024: "2025-04-13", 2023: "2024-04-14", 2022: "2023-04-09", 2021: "2022-04-10"}

df = pd.DataFrame()
for month in ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']:
    time.sleep(6)
    try:
        df_temp = pd.read_html(f"https://www.basketball-reference.com/leagues/NBA_{year + 1}_games-{month}.html")[0]
    except:
        pass
    df = pd.concat([df, df_temp])
df = df[['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1', 'Unnamed: 7', 'Arena', 'Notes']]\
        .rename(columns={"Visitor/Neutral": "AwayTeam", "Home/Neutral": "HomeTeam", "Start (ET)": "StartTime_ET", "Unnamed: 7": "is_OT"})
df['Date'] = pd.to_datetime(df.Date)

df_teams = pd.read_csv("../src/team_info_xref.csv")
df = con.execute(f"""SELECT Date, StartTime_ET, AwayTeam, ABV as AwayABV, PTS as AwayPTS, HomeTeam, HomeABV, "PTS.1" as HomePTS, is_OT, Arena, Notes FROM
                    (SELECT * EXCLUDE(ABV), ABV as HomeABV FROM df JOIN df_teams ON df.HomeTeam = df_teams.Team) t1
                    JOIN df_teams ON t1.AwayTeam = df_teams.Team
                    """).fetchdf().drop_duplicates()

# Create B2B columns
home = df[['Date', 'HomeABV']].rename(columns={'HomeABV':'Team'})
away = df[['Date', 'AwayABV']].rename(columns={'AwayABV':'Team'})
games = pd.concat([home, away], ignore_index=True).drop_duplicates()
games['Date'] = pd.to_datetime(games.Date)
games = games.sort_values(['Team', 'Date'])
games['prev_date'] = games.groupby('Team')['Date'].shift(1)
games['is_b2b'] = (games['Date'] - games['prev_date']).dt.days.eq(1).astype(int)
df = df.merge(
    games.rename(columns={'Team': 'AwayABV', 'is_b2b': 'AwayB2B'})[['Date', 'AwayABV', 'AwayB2B']],
    on=['Date', 'AwayABV'],
    how='left'
)
df = df.merge(
    games.rename(columns={'Team': 'HomeABV', 'is_b2b': 'HomeB2B'})[['Date', 'HomeABV', 'HomeB2B']],
    on=['Date', 'HomeABV'],
    how='left'
)

df['cup_gm'] = np.where(df.Notes.isin(['NBA Cup', 'In-Season Tournament']), 1, 0)
df['pstszn_gm'] = np.where(df.Date <= last_reg_gm[year], 0, 1)
df['rtrvd'] = 0
df = df.drop('Notes', axis=1)

df['is_OT'] = np.where(df.is_OT.isnull(), 0, df.is_OT)
df['is_OT'] = np.where(df.is_OT == 'OT', 1, df.is_OT)
df['is_OT'] = np.where(df.is_OT == '2OT', 2, df.is_OT)
df['is_OT'] = np.where(df.is_OT == '3OT', 3, df.is_OT)
df['is_OT'] = np.where(df.is_OT == '4OT', 4, df.is_OT)

# Fails if file doesn't exist
try:
    df = add_new_gms(df)
except:
    pass

df.to_csv(f"../tables/{year}/nba_schedule.csv", index=False)
display(df)

,Date,StartTime_ET,AwayTeam,AwayABV,AwayPTS,HomeTeam,HomeABV,HomePTS,is_OT,Arena,AwayB2B,HomeB2B,cup_gm,pstszn_gm,rtrvd
0,2025-10-21,7:30p,Houston Rockets,HOU,124.0,Oklahoma City Thunder,OKC,125.0,2,Paycom Center,0,0,0,0,1
1,2025-10-21,10:00p,Golden State Warriors,GSW,119.0,Los Angeles Lakers,LAL,109.0,0,Crypto.com Arena,0,0,0,0,1
2,2025-10-22,7:00p,Brooklyn Nets,BRK,117.0,Charlotte Hornets,CHO,136.0,0,Spectrum Center,0,0,0,0,1
3,2025-10-22,7:00p,Cleveland Cavaliers,CLE,111.0,New York Knicks,NYK,119.0,0,Madison Square Garden (IV),0,0,0,0,1
4,2025-10-22,7:00p,Miami Heat,MIA,121.0,Orlando Magic,ORL,125.0,0,Kia Center,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2026-04-12,8:30p,Utah Jazz,UTA,NaN,Los Angeles Lakers,LAL,NaN,0,Crypto.com Arena,0,0,0,0,0
1227,2026-04-12,8:30p,New Orleans Pelicans,NOP,NaN,Minnesota Timberwolves,MIN,NaN,0,Target Center,0,0,0,0,0
1228,2026-04-12,8:30p,Phoenix Suns,PHO,NaN,Oklahoma City Thunder,OKC,NaN,0,Paycom Center,0,0,0,0,0
1229,2026-04-12,8:30p,Sacramento Kings,SAC,NaN,Portland Trail Blazers,POR,NaN,0,Moda Center,0,0,0,0,0
